# Example Code for Estimating the Ground State Energy of Hydroxyl (·OH)

## Basic Installation

Install required package, we highly recommend participant to use qiskit platform, or at least participants can finish preprocessing at other platform and transfer the circuit to qiskit format, since our noise model is from IBM real machine backend and we restricted some algorithmic seeds which could be varied from different platform.

In [ ]:
!pip install qiskit
!pip install qiskit-nature[pyscf] -U

In [ ]:
!pip install qiskit_aer

In [34]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
import numpy as np
import pylab
import qiskit.providers
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as seeds that requires to run, and the result will be calculated as the average of results from each seed.

In [13]:
seed = 170
algorithm_globals.random_seed = seed
iterations = 125

## Generate Hamiltonian and Pauli String

At this step, the example code uses PySCF to generate the hamiltonian of hydroxyl with basis function as 'sto3g' to fit the spin orbital, then uses JordanWignerMapper to map the fermionic terms to pauli strings. To be noticed, other chemistry tool also allowed to be used at this step, but keep in mind to use 'sto-3g' and Jordan Wigner Mapper which should gives 12 qubits and 631 paulil terms.

In [3]:
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""

driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()

/usr/local/lib/python3.10/dist-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [ ]:
hamiltonian = qmolecule.hamiltonian
coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)
second_q_op = hamiltonian.second_q_op()

In [ ]:
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
qubit_op = converter.convert(second_q_op)

We recommend to use classical minimum eigensolver to obtain a reference energy at this step. In case some of the classical minimum eigensolver donot directly gives nuclear repulsion energy, we give reference energies below: *Comupted Energy*: -78.75252123, *Nuclear Repulsion_energy*: 4.36537496654537. *Obtained Reference Ground State Energy*: -74.3871462635.

In [6]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [15]:
result = solver.solve(qmolecule)
print(result.computed_energies)

[-78.75252123]


In [17]:
print(result.nuclear_repulsion_energy)

4.36537496654537


In [18]:
ref_value = result.computed_energies + result.nuclear_repulsion_energy

## Construct Ansatz

At this stage, you can implement various techniques to search good ansatz architecture which is important for variational quantum algorihms. Moreover, how to obtain a good initial state is a good topic to do research, we require participant to self-reflection there techniques (include the techniques for preprocessing ansatz or initial states) with maximum 10 points, and submit a short description for used techniques, we will have three graders to evaluate the techniques.

In [35]:
ansatz = UCCSD(
    qmolecule.num_spatial_orbitals,
    qmolecule.num_particles,
    mapper,
    initial_state=HartreeFock(
        qmolecule.num_spatial_orbitals,
        qmolecule.num_particles,
        mapper,
    ),
)


In [37]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

In [36]:
vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [39]:
res = calc.solve(qmolecule)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -78.318374453615
  - computed part:      -78.318374453615
~ Nuclear repulsion energy (Hartree): 4.365374966545
> Total ground state energy (Hartree): -73.952999487069
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 8.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.85037676  -0.28818323  -1.59757447]
 
  0: 
  * Electronic dipole moment (a.u.): [0.39515867568  -0.133932189247  -0.742341180385]
    - computed part:      [0.39515867568  -0.133932189247  -0.742341180385]
  > Dipole moment (a.u.): [0.45521808432  -0.154251040753  -0.855233289615]  Total: 0.981040706358
                 (debye): [1.157048850129  -0.392067001471  -2.173785990552]  Total: 2.493556517897
 
